In [269]:
import pandas as pd

In [374]:
df = pd.read_csv('all_data_not_advanced.csv', sep=',')

In [375]:
df.drop(['Unnamed: 0'], axis=1, inplace = True)

In [317]:
pd.set_option('display.max_columns', 200)
#df1 = df.drop(['Name','name','result','index','year_x','year_y','winner'], axis=1)
df1 = df.drop(['name','index','year_x','winner'], axis=1)

In [318]:
pd.set_option('display.max_row', 200)


In [319]:
df1 = df1.drop(['Name','year_y','result'], axis=1)

In [320]:
blah=split_all_data_up (df,df1['point_difference'])
df1['round']=blah
df1=df1[(df1['round']==2)]

In [321]:
df1=df1[(df1['team_seed']== 1) | (df1['team_seed']== 8)]

In [378]:
df1 = df1.drop(['round'], axis = 1)

KeyError: "['round' 'name' 'index' 'year_x' 'winner'] not found in axis"

In [302]:
df1['point_difference'].unique()

array([15,  1,  7,  3,  8,  4,  6, 11,  2,  0,  5, 10,  9, 12])

# Gets the opponent data

In [472]:
#pass through the dataframe you want to add to and the column name
#it will create a new column with the columnname_opponent
def get_opponent_data (df, column_name):
    first_data = []
    second_data = []
    i=0
    x = 0
    for thing in df[f'{column_name}']:
        if i ==0:
            first_data.append(df[f'{column_name}'].iloc[x+1])
            i+=1
            x+=1
        elif i==1:
            second_data.append(df[f'{column_name}'].iloc[x-1])
            i=0
            x+=1
    
    i=0
    x=0
    opponent_data = []
    for thing in first_data+second_data:
        if i==0:
            opponent_data.append(first_data[x])
            i+=1
        elif i==1:
            opponent_data.append(second_data[x])
            x+=1
            i=0
    print('made it here')        
    df[f'{column_name}'+'_opponent']=opponent_data


In [470]:
get_opponent_data (df1, 'PTS')

made it here


# Decision Tree

In [323]:
X = df1.drop('win',axis=1)
#could have also done answers1.set_index('SEQN') instead of dropping
y = df1['win']

In [293]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


In [390]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 123)


In [325]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [326]:
predictions = dtree.predict(X_test)


In [327]:
print(classification_report(y_test,predictions))


              precision    recall  f1-score   support

           0       0.82      0.82      0.82        22
           1       0.90      0.90      0.90        41

    accuracy                           0.87        63
   macro avg       0.86      0.86      0.86        63
weighted avg       0.87      0.87      0.87        63



In [298]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions)


0.7217741935483871

In [384]:

from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydot 

features = list(X_train.columns)
len(features)

20

In [434]:
#function for accuracy in decision tree
def get_predictions (point_diff):
    df = pd.read_csv('all_data_not_advanced.csv', sep=',')
    df.drop(['Unnamed: 0'], axis=1, inplace = True)
    pd.set_option('display.max_columns', 200)
    #df1 = df.drop(['Name','name','result','index','year_x','year_y','winner'], axis=1)
    df1 = df.drop(['name','index','year_x','winner'], axis=1)
    #pd.set_option('display.max_row', 200)
    df1 = df1.drop(['Name','year_y','result'], axis=1)
    blah=split_all_data_up (df,df1['point_difference'])
    df1['round']=blah
    #unhashtag this if you want the ESPN stuff also
    #df1=df1[['point_difference','team_seed','win','higher_or_lower_seed','other_team_seed','round']]
    #unhashtag the row below if you want to look at a specific round
    #df1=df1[(df1['round']==4)]
    df1=df1[(df1['point_difference']== point_diff)]
    #df1 = df1.drop(['round'], axis = 1)
    X = df1.drop('win',axis=1)
    #could have also done answers1.set_index('SEQN') instead of dropping
    y = df1['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    dtree = DecisionTreeClassifier()
    dtree.fit(X_train,y_train)
    predictions = dtree.predict(X_test)
    #print(classification_report(y_test,predictions))

    return metrics.accuracy_score(y_test, predictions)


In [435]:
predictions = {}
for num in possible_point_differences:
    try:
        predictions[f'{num}']=get_predictions (num)
    except:
        continue

In [429]:
fourth_round_pred_no_stats = predictions

In [436]:
#with stats
predictions

{'15': 0.9761904761904762,
 '1': 0.5067873303167421,
 '7': 0.5294117647058824,
 '3': 0.536723163841808,
 '8': 0.6326530612244898,
 '4': 0.4788732394366197,
 '6': 0.6363636363636364,
 '11': 0.7604166666666666,
 '2': 0.5806451612903226,
 '0': 1.0,
 '5': 0.54421768707483,
 '10': 0.6,
 '9': 0.6428571428571429,
 '12': 1.0}

In [433]:
#no stats
predictions

{'15': 1.0,
 '1': 0.5158371040723982,
 '7': 0.7058823529411765,
 '3': 0.655367231638418,
 '8': 0.7551020408163265,
 '4': 0.7183098591549296,
 '6': 0.5454545454545454,
 '11': 0.84375,
 '2': 0.5806451612903226,
 '0': 1.0,
 '5': 0.6462585034013606,
 '10': 0.4,
 '9': 0.7448979591836735,
 '12': 1.0}

In [414]:
All_pred_no_stats

{'15': 1.0,
 '1': 0.5203619909502263,
 '7': 0.6838235294117647,
 '3': 0.6214689265536724,
 '8': 0.7619047619047619,
 '4': 0.7323943661971831,
 '6': 0.7272727272727273,
 '11': 0.8541666666666666,
 '2': 0.45161290322580644,
 '0': 1.0,
 '5': 0.673469387755102,
 '10': 0.2,
 '9': 0.7857142857142857,
 '12': 1.0}

In [418]:
first_round_pred_no_stats

{'15': 0.9880952380952381,
 '1': 0.42857142857142855,
 '7': 0.5833333333333334,
 '3': 0.5476190476190477,
 '11': 0.8690476190476191,
 '5': 0.6428571428571429,
 '9': 0.7142857142857143}

In [366]:
first_round_pred = predictions

In [367]:
first_round_pred

{'15': 0.9880952380952381,
 '1': 0.5238095238095238,
 '7': 0.6785714285714286,
 '3': 0.44047619047619047,
 '11': 0.6904761904761905,
 '5': 0.5119047619047619,
 '9': 0.6309523809523809}

In [422]:
second_round_pred_no_stats

{'1': 0.5294117647058824,
 '7': 0.8095238095238095,
 '3': 0.7021276595744681,
 '8': 0.7602739726027398,
 '5': 0.6792452830188679}

In [359]:
second_round_pred

{'1': 0.5686274509803921,
 '7': 0.6190476190476191,
 '3': 0.574468085106383,
 '8': 0.589041095890411,
 '5': 0.5849056603773585}

In [426]:
third_round_pred_no_stats

{'1': 0.5757575757575758,
 '7': 0.5555555555555556,
 '3': 0.7352941176470589,
 '4': 0.6716417910447762,
 '11': 1.0,
 '5': 0.6666666666666666,
 '9': 0.7777777777777778,
 '12': 1.0}

In [360]:
third_round_pred

{'1': 0.45454545454545453,
 '7': 0.4444444444444444,
 '3': 0.5588235294117647,
 '4': 0.5223880597014925,
 '11': 1.0,
 '5': 0.3333333333333333,
 '9': 0.6666666666666666,
 '12': 1.0}

In [430]:
fourth_round_pred_no_stats

{'1': 0.3939393939393939,
 '7': 0.0,
 '3': 0.5,
 '6': 0.875,
 '2': 0.4583333333333333,
 '5': 0.8571428571428571,
 '10': 0.4,
 '9': 1.0}

In [361]:
fourth_round_pred

{'1': 0.45454545454545453,
 '7': 0.0,
 '3': 0.5,
 '6': 1.0,
 '2': 0.2916666666666667,
 '5': 0.5714285714285714,
 '10': 0.2,
 '9': 0.3333333333333333}

In [362]:
fifth_round_pred

{'1': 0.5833333333333334,
 '7': 0.0,
 '3': 0.6666666666666666,
 '8': 1.0,
 '4': 0.3333333333333333,
 '6': 0.3333333333333333,
 '2': 0.6666666666666666,
 '0': 1.0,
 '9': 0.5}

In [363]:
sixth_round_pred

{'1': 0.4444444444444444,
 '7': 0.0,
 '3': 0.0,
 '4': 1.0,
 '2': 0.5,
 '0': 1.0,
 '5': 0.5}

In [308]:
All_rounds_predictions

{'15': 1.0,
 '1': 0.5067873303167421,
 '7': 0.5882352941176471,
 '3': 0.5141242937853108,
 '8': 0.6394557823129252,
 '4': 0.5915492957746479,
 '6': 0.6363636363636364,
 '11': 0.7916666666666666,
 '2': 0.3870967741935484,
 '0': 1.0,
 '5': 0.5238095238095238,
 '10': 0.8,
 '9': 0.7142857142857143,
 '12': 1.0}

In [337]:
possible_point_differences=[15,  1,  7,  3,  8,  4,  6, 11,  2,  0,  5, 10,  9, 12]

# Insert rounds back in

In [76]:
def split_all_data_up (df,column):
    round_=[]
    i=0
    for thing in list(column):
        if i== 62:
            round_.append(6)
            round_.append(6)
            i=0
        elif 0<=i<=7 or 15<=i<=22 or 30<=i<=37 or 45<=i<=52:
            round_.append(1)
            round_.append(1)
            i+=1
        elif 8<=i<=11 or 23<=i<=26 or 38<=i<=41 or 53<=i<=56:
            round_.append(2)
            round_.append(2)
            i+=1
        elif 12<=i<=13 or 27<=i<=28 or 42<=i<=43 or 57<=i<=58:
            round_.append(3)
            round_.append(3)
            i+=1
        elif i==14 or i==29 or i==44 or i==59:
            round_.append(4)
            round_.append(4)
            i+=1
        elif i == 60 or i == 61:
            round_.append(5)
            round_.append(5)
            i+=1
    round_ = round_[0:4410]
    return round_

In [122]:
blah=split_all_data_up (df,df1['point_difference'])

In [123]:
len(blah)

4410

In [124]:
df1['round']=blah